# Exercício
Desenvolva um pequeno programa que crie documentos e salve em uma coleção e siga os seguintes pontos:

1. O documento deve possuir dois campos (val1 e val2) numéricos com valores aleatórios de 0 a 100.
1. Gere e insira pelo menos 1 milhão de documentos (meça o tempo de inserção total).
1. Realize uma consulta por valores em val1 entre 0 e 10 e meça o tempo.
1. Crie um índice pelo campo val1.
1. Repita a consulta anterior medindo o tempo. O que ocorre?
1. Agora repita a consulta anterior retornando apenas o campo val1 (utilize projeção para remover o _id e val2). O que ocorre?
1. Insira mais 1 milhão de registros.

Docker com o MongoDB:

```bash
docker run -p 27017:27017 --name exercicio-mongo -v /home/mongo:/data/ -d mongo
```

# Resolução

In [8]:
import pymongo
from random import randrange

In [2]:
# MongoClient
client = pymongo.MongoClient('mongodb://localhost:27017/')
# Database
db = client['exercicio_mongo']
# Collection
indexes = db['indexes']

In [3]:
client.server_info()

{'version': '4.2.0',
 'gitVersion': 'a4b751dcf51dd249c5865812b390cfd1c0129c30',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 0, 0],
 'openssl': {'running': 'OpenSSL 1.1.1  11 Sep 2018',
  'compiled': 'OpenSSL 1.1.1  11 Sep 2018'},
 'buildEnvironment': {'distmod': 'ubuntu1804',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0',
  'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp',
  'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.2.0',
  'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-deallocation -std=c++17',
  'linkflags': '-pthread -Wl,-z,now -rdynamic -Wl,--fatal-war

1. O documento deve possuir dois campos (val1 e val2) numéricos com valores aleatórios de 0 a 100.

In [4]:
# Quantidade de documentos
docs_num = 1000000
# Documentos gerados
indexes_data = [
    {
        'val1': randrange(100),
        'val2': randrange(100),
    } for i in range(docs_num)
]

2. Gere e insira pelo menos 1 milhão de documentos (meça o tempo de inserção total).

In [5]:
%%time
# Inserção
indexes.insert_many(indexes_data)

CPU times: user 8.54 s, sys: 232 ms, total: 8.77 s
Wall time: 12.6 s


3. Realize uma consulta por valores em val1 entre 0 e 10 e meça o tempo.

In [6]:
%%time
cursor = indexes.find({
    'val1': {
        '$gte': 0,
        '$lte': 10,
    }
})

for c in cursor:
    pass

CPU times: user 460 ms, sys: 4 µs, total: 460 ms
Wall time: 936 ms


4. Crie um índice pelo campo val1.

In [9]:
# Criando index
indexes.create_index([('val1', pymongo.ASCENDING)])

'val1_1'

5. Repita a consulta anterior medindo o tempo. O que ocorre?

In [10]:
%%time
# Busca
cursor = indexes.find({
    'val1': {
        '$gte': 0,
        '$lte': 10,
    }
})

for c in cursor:
    pass

CPU times: user 257 ms, sys: 4.02 ms, total: 261 ms
Wall time: 458 ms


6. Agora repita a consulta anterior retornando apenas o campo val1 (utilize projeção para remover o _id e val2). O que ocorre?

In [11]:
%%time
# Busca com apenas o campo val1
cursor = indexes.find({
    'val1': {
        '$gte': 0,
        '$lte': 10,
    }
}, {
    '_id': 0,
    'val2': 0,
})

for c in cursor:
    pass

CPU times: user 168 ms, sys: 69 µs, total: 168 ms
Wall time: 432 ms


7. Insira mais 1 milhão de registros.

In [12]:
# Novos documentos gerados
indexes_data_new = [
    {
        'val1': randrange(100),
        'val2': randrange(100),
    } for i in range(docs_num)
]

In [13]:
%%time
# Inserção
indexes.insert_many(indexes_data_new)

CPU times: user 9.41 s, sys: 300 ms, total: 9.71 s
Wall time: 16.8 s


# Resumo
Valores do _time_ antes e após a indexação.

| Operação | Antes da indexação | Depois da indexação |
| -------- | ----- | ------ |
| Inserção | 9.01 s |14.1 s |
| Busca | 1.21 s | 464 ms |
| Busca com Projeção | - | 423 ms |

In [14]:
for i in indexes.find()[:10]:
    print(i)

{'_id': ObjectId('5da717a5429696fe514789bc'), 'val1': 1, 'val2': 12}
{'_id': ObjectId('5da717a5429696fe514789bd'), 'val1': 5, 'val2': 27}
{'_id': ObjectId('5da717a5429696fe514789be'), 'val1': 35, 'val2': 92}
{'_id': ObjectId('5da717a5429696fe514789bf'), 'val1': 58, 'val2': 90}
{'_id': ObjectId('5da717a5429696fe514789c0'), 'val1': 57, 'val2': 28}
{'_id': ObjectId('5da717a5429696fe514789c1'), 'val1': 67, 'val2': 97}
{'_id': ObjectId('5da717a5429696fe514789c2'), 'val1': 27, 'val2': 65}
{'_id': ObjectId('5da717a5429696fe514789c3'), 'val1': 79, 'val2': 76}
{'_id': ObjectId('5da717a5429696fe514789c4'), 'val1': 42, 'val2': 0}
{'_id': ObjectId('5da717a5429696fe514789c5'), 'val1': 53, 'val2': 69}
